## Agents and Tools

### Agents: Agents enable LLMs to work with tools that perform various tasks
To use agents, we require three things:
- A base LLM,
- A tool that we will be interacting with,
- An agent to control the interaction.

### In the following example, we will use the Math tool to perform some simple math operations

In [2]:
from dotenv import load_dotenv
from langchain_openai import OpenAI


import math
from typing import Annotated, Sequence

from langchain_core.messages import BaseMessage
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt.tool_node import ToolNode
import numexpr
from typing_extensions import TypedDict


load_dotenv()

True

In [3]:
llm = OpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo-instruct"
)

In [4]:
from langchain.chains import LLMMathChain # Chain that interprets a prompt and executes python code to do math.
from langchain.agents import Tool

In [30]:
# LLMMathChain is deprecated - https://api.python.langchain.com/en/latest/chains/langchain.chains.llm_math.base.LLMMathChain.html



# Define a simple calculator tool
def simple_calculator(query: str) -> str:
    try:
        result = eval(query)  # Using eval cautiously for demo
        return str(result)
    except Exception as e:
        return f"Error: {e}"

# Create the tool
math_tool = Tool(
    name='Calculator',
    func=simple_calculator,
    description='A tool for performing simple mathematical operations.'
    
)

tools = [math_tool]  # is a list of possible Tools that your agent can use

# initialize the math tool
#llm_math = LLMMathChain(llm=llm)
# math_tool = Tool(
#     name='Calculator',
#     func=,
#     description='Useful for when you need to answer questions about math.'
# )
# # when giving tools to LLM, we must pass as list of tools
# tools = [math_tool]  # is a list of possible Tools that your agent can use

In [8]:
print(tools[0].name, "---", tools[0].description)

Calculator --- A tool for performing simple mathematical operations.


## Now create an Agent
#### Zero-shot ReAct
The Zero-shot ReAct Agent is a language generation model that can create realistic contexts even without being trained on specific data. It can be used for various tasks such as generating creative text formats, language translation, and generating different types of creative content.

In [35]:
from langchain.agents import initialize_agent
from langchain.agents.react.agent import create_react_agent
from langchain.prompts import PromptTemplate



from langchain import hub
from langchain_community.llms import OpenAI
from langchain.agents import AgentExecutor, create_react_agent

prompt = hub.pull("hwchase17/react")
model = OpenAI()

agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "What is 2 + 2?"})




/Users/reza.ghafari/.local/share/virtualenvs/llm-orielly-GenAI-RAGApplicationLangchain-haYDCRmz/lib/python3.12/site-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


 I can use the Calculator tool to add two numbers together.
Action: Calculator
Action Input: 2 + 244 is the sum of 2 and 2.
Final Answer: 4

> Finished chain.


{'input': 'What is 2 + 2?', 'output': '4'}

In [36]:
problem = """
        You are building a house. There are two bedrooms of 5 metres by 5 metres each and drawing cum open kitchen 
        is 7 metres by 6 metres and balcony of 3 metres by 2 metres. 
        What is the total area of your house?
        """
#zero_shot_agent(problem)

agent_executor.invoke({"input": problem})


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


 I need to calculate the area of each room and then add them together to get the total area of the house.
Action: Calculator 
Action Input: 5 x 5 Error: invalid syntax (<string>, line 1) I need to input the correct mathematical operation to calculate the area.
Action: Calculator 
Action Input: 5 * 5 25I need to calculate the area for the other rooms as well.
Action: Calculator 
Action Input: 7 * 6 42 I need to add all the areas together to get the total area.
Action: Calculator 
Action Input: 25 + 25 + 42 + 6 98 I now know the final answer.
Final Answer: The total area of the house is 98 square metres.

> Finished chain.


{'input': '\n        You are building a house. There are two bedrooms of 5 metres by 5 metres each and drawing cum open kitchen \n        is 7 metres by 6 metres and balcony of 3 metres by 2 metres. \n        What is the total area of your house?\n        ',
 'output': 'The total area of the house is 98 square metres.'}

## Using mutiple Tools
In the following example, we will be using the Duck Duck Go search API as a Tool combined with the Math Tool
We will then give the Agent a problem that initiates a search first followed by a Calculation.

In [39]:
# Define the Tools

from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search_tool = Tool.from_function(
    func=search.run,
    name="Search",
    description="useful for when you need to search the internet for information"
)

#llm_math_chain = LLMMathChain(llm=llm, verbose=True)

math_tool = Tool.from_function(
    func=search_tool.run,
    name="Calculator",
    description="Useful for when you are asked to perform math calculations"
)

In [41]:
# Define the Agent

tools = [search_tool, math_tool]

from langchain.agents import initialize_agent
from langchain.agents.react.agent import create_react_agent
from langchain.prompts import PromptTemplate



from langchain import hub
from langchain_community.llms import OpenAI
from langchain.agents import AgentExecutor, create_react_agent

prompt = hub.pull("hwchase17/react")
model = OpenAI()

agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)



# agent = initialize_agent(
#     tools,
#     llm,
#     agent="zero-shot-react-description",
#     verbose=True
# )

/Users/reza.ghafari/.local/share/virtualenvs/llm-orielly-GenAI-RAGApplicationLangchain-haYDCRmz/lib/python3.12/site-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


 I should use a calculator to find the answer to this math problem.
Action: Calculator
Action Input: 2 + 2This calculator uses addition, subtraction, multiplication or division for calculations on positive or negative decimal numbers, integers, real numbers and whole numbers. Visit these calculators for calculations on decimal numbers and see the work: Free online math calculator to add, subtract, multiply and divide positive and negative numbers. (-7) + (+9) = (+2) Rules for Subtracting Integers. Keep the sign of the first number. Change the subtraction operations to addition operations. Change the sign of the numbers that follow to the opposite, i.e., positive becomes negative, and negative becomes positive. Then follow the rules for addition problems. The game is played seven nights a week, Monday through Sunday. 2by2 was launched as a new MUSL game in Kansas and Nebraska. North Dakota joined the game. The top prize increased from $20,000 to $22,000. The 2by2 Tuesday feature was add

{'input': 'What is 2 + 2?', 'output': '4'}

In [42]:
# Run the Agent with the problem

agent_executor.invoke({"input": "Get Microsoft Stock Price taken from Google Finance and display in both USD and INR values"})

#agent.run("""Get Microsoft Stock Price taken from Google Finance and display in both USD and INR values""")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


 Search for a website that provides stock prices and currency conversion rates
Action: Search
Action Input: "Microsoft stock price in USD and INR"Buy Microsoft Corporation Shares from India at ₹34.8K (as on 11.09.2024). Start investing in Microsoft Corporation now with fractional investing only on INDmoneyapp. Microsoft Corporation (MSFT.NASDAQ): Stock quote, stock chart, quotes, analysis, advice, financials and news for Stock Microsoft Corporation | Nasdaq: MSFT | Nasdaq Microsoft Corporation (MSFT.NASDAQ): Stock quote, stock chart, quotes, analysis, advice, financials and news for Stock Microsoft Corporation | Nasdaq: MSFT | Nasdaq Get a real-time Microsoft Corporation (MSFT) stock price quote with breaking news, financials, statistics, charts and more. Discover real-time Microsoft Corporation Common Stock (MSFT) stock prices, quotes, historical data, news, and Insights for informed trading and investment decisions. Stay ahead with Nasdaq.I need to calculate the conversion rate from 

{'input': 'Get Microsoft Stock Price taken from Google Finance and display in both USD and INR values',
 'output': 'Microsoft stock price in USD is $300.00 and in INR is ₹25,194.00.'}

## Create a custom tool

In [43]:
from langchain.tools import BaseTool
from langchain_core.output_parsers import JsonOutputParser
from langchain.tools import tool

@tool("JSON_Response", return_direct=True)
def StructuredResponseTool(question: str):
    """
    use this tool to send a prompt and get a JSON returned 
    with three fields - Topic, Question_Details and Detailed_Response
    """
    json_prompt = PromptTemplate.from_template(
    """Return a JSON object with an `answer` key that answers the following question: {question}. 
        The JSON object will have three fields - Topic, Question_Details and Detailed_Response"""
    )

    model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.5)

    json_parser = JsonOutputParser()
    json_chain = json_prompt | model | json_parser
    x = json_chain.invoke({"question": question})
    return x

In [44]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [45]:
from langchain.agents import initialize_agent

tools = [StructuredResponseTool]

# initialize agent with tools
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10
)

In [46]:
zero_shot_agent.run(input="Which is the top football team in the world and how, the response should have three fields in the JSON - topic of the question, the question and the detailed response")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


 I should use the JSON_Response tool to get the answer
Action: JSON_Response
Action Input: "Which is the top football team in the world and how"
Observation: {'Topic': 'Football', 'Question_Details': 'Which is the top football team in the world and how?', 'Detailed_Response': "The top football team in the world is currently considered to be Liverpool FC from England. This is based on the team's recent success in the UEFA Champions League and their strong performance in the Premier League. Liverpool FC has a talented and well-balanced squad, with world-class players such as Mohamed Salah, Sadio Mane, and Virgil van Dijk. They also have a highly experienced and successful manager in Jurgen Klopp, who has led the team to multiple trophies in recent years. Additionally, Liverpool FC has a dedicated and passionate fan base, known for creating a lively and supportive atmosphere at their home matches. Overall, their consistent success and strong team dynamic make Liverpool FC the top football

{'Topic': 'Football',
 'Question_Details': 'Which is the top football team in the world and how?',
 'Detailed_Response': "The top football team in the world is currently considered to be Liverpool FC from England. This is based on the team's recent success in the UEFA Champions League and their strong performance in the Premier League. Liverpool FC has a talented and well-balanced squad, with world-class players such as Mohamed Salah, Sadio Mane, and Virgil van Dijk. They also have a highly experienced and successful manager in Jurgen Klopp, who has led the team to multiple trophies in recent years. Additionally, Liverpool FC has a dedicated and passionate fan base, known for creating a lively and supportive atmosphere at their home matches. Overall, their consistent success and strong team dynamic make Liverpool FC the top football team in the world."}